In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats
import math
pd.set_option('display.float_format', lambda x: '%.2f' % x)

def helperWeibull(x, y, vec_cum, n):
        """
            :param x: x - a list of media values (ex facebook_i)
            :param y: x_bin - a list 1:n with n being number of values in period
            :param vec_cum: thetaVecCum - the cumulative theta vector
            :param n: the number of values in the media channel (X days, weeks, etc)
            :return: numpy
        """
        ## Initializing a vector with 0's padded at the beginning
        ## ie if you're on the 3rd day of 5 days and the impressions for that day is 100 then x.vec will be
        ## c(0, 0, 100, 100, 100)
        x_vec = np.array([0] * (y - 1) + [x] * (n - y + 1))
        
        # Shift the cumulative theta values by 1 each iteration and pads beginning with 0's
        # This makes the theta value to start as 1 for each new day
        # If vec_cum is [1.0, .6, .5, .03, .01], then at:
            # y=1 -> [1.0, .6, .5, .03, .01]
            # y=2 -> [0.0, 1.0, .6, .5, .03]
            # y=3 -> [0.0, 0.0, 1.0, .6, .5]
        vec_lag = np.roll(vec_cum, y - 1)
        vec_lag[: y - 1] = 0
        
        # Lines up impressions with theta value for that period
        x_matrix = np.c_[x_vec, vec_lag]
        
        # Multiplies the number of impressions on that day by the theta value for that day
        x_prod = np.multiply.reduce(x_matrix, axis=1)
        return x_prod

In [4]:
## LOAD DATA FROM MODEL OUTPUT - ANY TRANSFORM CAN BE BACK-ENGINEERED

# rolling windowlength can be found by InputCollect$rollingWindowLength
windlen = 10

# Read in the hyperparameters for all the models then limit to 1 model
hp_path = 's3://mti-sandbox/robyn_demo_outputs/robyn_demo_weibull_cdf/pareto_hyperparameters.csv'
hyperparam_df = pd.read_csv(hp_path)
sol_id = hyperparam_df['solID'].unique()[1]

# Get the values of the hyperparameters for one channel for POC
hyperparams_fb = hyperparam_df[hyperparam_df['solID'] == sol_id][[c for c in hyperparam_df.columns if 'facebook' in c]]
print(f'>> HYPERPARAMETER VALUES FOR MODEL {sol_id}:')
display(hyperparams_fb)

# Load media transform values and limit to selected model
mt_path = 's3://mti-sandbox/robyn_demo_outputs/robyn_demo_weibull_cdf/pareto_media_transform_matrix.csv'
media_transform_df = pd.read_csv(mt_path)
media_transform_df = media_transform_df[media_transform_df['solID'] == sol_id] \
                    .sort_values(by='ds')[['ds', 'facebook_I', 'type', 'solID']]

print(f'>> MEDIA TRANSFORMS:')
cdf_sample = media_transform_df[media_transform_df['type'].isin(['rawMedia', 'adstockedMedia'])][['ds', 'type','facebook_I']]
display(cdf_sample.loc[[0,75,1,76,2,77,3,78]])

>> HYPERPARAMETER VALUES FOR MODEL 1_1_2:


,facebook_I_alphas,facebook_I_gammas,facebook_I_scales,facebook_I_shapes
1,1.52,0.74,0.09,0.89


>> MEDIA TRANSFORMS:


,ds,type,facebook_I
0,2019-04-22,rawMedia,19361443.73
75,2019-04-22,adstockedMedia,19361443.73
1,2019-04-29,rawMedia,31687579.01
76,2019-04-29,adstockedMedia,42994997.40
2,2019-05-06,rawMedia,0.00
77,2019-05-06,adstockedMedia,22665861.59
3,2019-05-13,rawMedia,65207240.82
78,2019-05-13,adstockedMedia,73003423.71


In [5]:
## LOAD DATA FROM MODEL OUTPUT - ANY TRANSFORM CAN BE BACK-ENGINEERED

# rolling windowlength can be found by InputCollect$rollingWindowLength
windlen = 15

# Get the values of the hyperparameters for one channel for POC
hyperparams_fb = hyperparam_df[hyperparam_df['solID'] == sol_id][[c for c in hyperparam_df.columns if 'facebook' in c]]
print(f'>> HYPERPARAMETER VALUES FOR MODEL {sol_id}:')
display(hyperparams_fb)

# Load media transform values and limit to selected model
mt_path = 's3://mti-sandbox/robyn_demo_outputs/robyn_demo_weibull_cdf/pareto_media_transform_matrix.csv'
media_transform_df = pd.read_csv(mt_path)
media_transform_df = media_transform_df[media_transform_df['solID'] == sol_id] \
                    .sort_values(by='ds')[['ds', 'facebook_I', 'type', 'solID']]

print(f'>> MEDIA TRANSFORMS:')
cdf_sample = media_transform_df[media_transform_df['type'].isin(['rawMedia', 'adstockedMedia'])][['ds', 'type','facebook_I']]
display(cdf_sample.loc[[0,75,1,76,2,77,3,78]])

>> HYPERPARAMETER VALUES FOR MODEL 1_1_2:


,facebook_I_alphas,facebook_I_gammas,facebook_I_scales,facebook_I_shapes
1,1.52,0.74,0.09,0.89


>> MEDIA TRANSFORMS:


,ds,type,facebook_I
0,2019-04-22,rawMedia,19361443.73
75,2019-04-22,adstockedMedia,19361443.73
1,2019-04-29,rawMedia,31687579.01
76,2019-04-29,adstockedMedia,42994997.40
2,2019-05-06,rawMedia,0.00
77,2019-05-06,adstockedMedia,22665861.59
3,2019-05-13,rawMedia,65207240.82
78,2019-05-13,adstockedMedia,73003423.71
